In [9]:
from sklearn import *
import numpy as np
import pandas as pd
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import preprocessing

In [10]:
sys.path.append('../utilities/')
from utils import *

In [11]:
dat = pd.read_csv('../CSVData/04.13.2019 vs UVA 2.csv')

In [12]:
dat.columns

Index(['#', 'Name', 'ClipType', 'Media In', 'Media Out', 'Hit Location X',
       'Hit Location Y', 'Hit Location', 'Hit Result', 'At Bat Pitch Count',
       'Batter', 'Bunt', 'Count', 'Inning', 'Outs', 'Pitch Location X',
       'Pitch Location Y', 'RBI', 'Total Pitch Count', 'Umpire', 'Strike Type',
       'Pitcher', 'Home Score', 'Home Team', 'Hit Type', 'Batter Strength',
       'Away Team', 'Ball Type', 'Base', 'Pitch Count', 'Pitch Type',
       'Team in Field', 'Team at Bat', 'Visitor Score'],
      dtype='object')

In [13]:
testDat = subset(dat, ['Pitch Location X', 'Pitch Location Y', 'Strike Type', 'Ball Type', 'Umpire'])
testDat['Result'] = np.nan

In [14]:
for idx, ball in enumerate(testDat['Ball Type']):
    if(type(ball) == str):
        testDat['Result'][idx] = ball
    else:
        testDat['Result'][idx] = testDat['Strike Type'][idx]

/home/nmaynard/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/nmaynard/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/nmaynard/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate

In [15]:
testDat2 = testDat[~testDat['Result'].isin(['Ball in Play', 'Swing', 'Foul', np.nan])]
testDat2.Result.map({'Take': 1, 'Ball':0})
testDat2.columns = ['pitchX', 'pitchY', 'SType', 'BType', 'Umpire', 'Result']
testDat2

,pitchX,pitchY,SType,BType,Umpire,Result
0,51,97,Take,NaN,NaN,Take
1,46,76,Take,NaN,NaN,Take
2,35,127,NaN,Ball,NaN,Ball
6,67,96,Take,NaN,NaN,Take
7,41,1,NaN,Ball,NaN,Ball
8,52,86,Take,NaN,NaN,Take
9,132,204,NaN,Ball,NaN,Ball
10,34,147,NaN,Ball,NaN,Ball
13,124,59,Take,NaN,NaN,Take
14,172,92,Take,NaN,NaN,Take


In [9]:


X = np.array((testDat2['Pitch Location X'].astype(float)))
Y = np.array((testDat2['Pitch Location Y'].astype(float)))

vec = np.array([X, Y]).reshape(120, 2)

labels = np.array(testDat2['Result'])


KeyError: 'Pitch Location X'

In [10]:
sns.lmplot('Pitch Location X', 'Pitch Location Y', data=testDat2, hue='Result', fit_reg=False)

KeyError: "['Pitch Location Y', 'Pitch Location X'] not in index"

In [63]:
sns.lmplot('Pitch Location X', 'Pitch Location Y', data=testDat2, hue='Result2', fit_reg=False)

KeyError: "['Result2'] not in index"

In [50]:
def wTx(w, x):
    return np.dot(x, w)
    
def sigmoid(z):
    

In [ ]:
modfit = svm.SVR(kernel = 'poly')
modfit.fit(vec, labels)

/home/nmaynard/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
lab = modfit.predict(vec)
labels

testDat2['Result2'] = lab
testDat2


In [16]:

def make_meshgrid(ax, h=.02):
    # x_min, x_max = x.min() - 1, x.max() + 1
    # y_min, y_max = y.min() - 1, y.max() + 1
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out


def draw_boundary(ax, clf):

    xx, yy = make_meshgrid(ax)
    return plot_contours(ax, clf, xx, yy,cmap=plt.cm.coolwarm, alpha=0.5)

def plot_SVM(gamma=1, C=1):
    umpire = testDat2
    umpire.type = umpire.Result.map({'Take': 1, 'Ball':0})
    fig, ax = plt.subplots()
    plt.scatter(umpire.pitchX, umpire.pitchY, c = umpire.type, cmap = plt.cm.coolwarm, alpha=0.6)
    training_set, validation_set = train_test_split(umpire, random_state=1)
    classifier = SVC(kernel='poly', gamma = gamma, C = C)
    classifier.fit(training_set[['pitchX', 'pitchY']], training_set['Result'])
    draw_boundary(ax, classifier)
    plt.show()
    print("The score of SVM with gamma={0} and C={1} is:".format(gamma, C) )
    print(classifier.score(validation_set[['pitchX', 'pitchY']], validation_set['Result']))

In [ ]:
plot_SVM()

/home/nmaynard/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
